# <h1 style="font-size: 18px;"> Info</h1>

https://www.kaggle.com/datasets/dnkumars/industrial-equipment-monitoring-dataset?resource=download<br>
<span style="font-size: 16px;">
This dataset contains simulated data representing real-time monitoring of various industrial equipment, including turbines,<br> compressors, and pumps. Each row in the dataset corresponds to a unique observation capturing key parameters such as temperature,<br> pressure, vibration, and humidity. The dataset also includes information about the equipment type, location, and whether the equipment is classified as faulty.
</span>
<br>

<span style="font-size: 16px;">
- Columns Description:<br>
- temperature: Temperature reading at the time of observation (in °C).<br>
- pressure: Pressure reading at the time of observation (in bar).<br>
- vibration: Vibration level reading (normalized units).<br>
- humidity: Humidity percentage recorded at the location of the equipment.<br>
- equipment: Type of industrial equipment being monitored (e.g., Turbine, Compressor, Pump).<br>
- location: Location of the equipment (city name).<br>
- faulty: Binary indicator (0 = Not Faulty, 1 = Faulty) to specify whether the equipment is functioning properly or requires maintenance.<br>
</span>

# <h1 style="font-size: 18px;"> Industrial Analysis</h1>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import plotly.graph_objects as go
import plotly.express as px

In [2]:
df = pd.read_csv("equipment_anomaly_data.csv")

In [3]:

# Create a class to reduce the quantity of code
class Analysis:
  """
  Class to analyze the dataframe.
    
  Attributes:
  - proportion: Print proportion of each categorical column and dependent variable
  - plot_kde: Plot the density curve of each numeric column
  - plot_boxplot: Plot the BoxPlot of each numeric column
  - outliers: Returns a dict of outliers of each numeric column
  - correlation: Print the correlation between numeric columns and dependent variable
  - corr_heatmap: Plot a HeatMap of the Correlation
  - logistic_plot: Plot a ScatterPlot of each numeric column and dependent variable
  """
  def __init__(self, dataframe):
    self.data = dataframe
    
  # Proportion
  @property
  def proportion(self):
    """
      Return the proportion of categorical columns also the dependent variable
    """
    data = self.data
    for col in data[["equipment", "location", "faulty"]]:
      print(f"{data[col].value_counts(normalize=True)}\n")
  
  # Plot Density Curve
  @property
  def plot_kde(self):
    """Fuction to plot density curve
      
    Columns:
    - temperature
    - pressure
    - vibration
    - humidity
    """
    data = self.data
    for col in data[["temperature", "pressure", "vibration", "humidity"]]:
      plt.figure(figsize=(10,4))
      sns.kdeplot(data[col], fill=True)
      plt.axvline(
        data[col].median(),
        color="red",
        label=f"Median: {round(data[col].median(), 2)}",
        linestyle="dashed"
      )
      plt.axvline(
        data[col].mean(),
        color="green",
        label=f"Mean: {round(data[col].mean(), 2)}",
        linestyle="dashed"
      )
      plt.legend(loc=0)
      plt.show()

  # Plot Boxplot
  @property
  def plot_boxplot(self):
    """Plot the BoxPlot of the numeric columns
    """
    data = self.data
    for col in data.drop(columns=["equipment", "location", "faulty"]):
      fig = px.box(data, y=col)
      fig.update_layout(
        scene=dict(
          xaxis_title='Temperature',
          yaxis_title='Pressure',
          zaxis_title='Vibration'
        ),
        width=1000, height=800
      )
      fig.show()
  
  # Método do Quartil.
  @property
  def outliers(self):
    """Return a dict with the outliers of each numeric column

    Returns:
        dict: variable name:outliers
    """
    outliers = {}
    data = self.data
    for col in data.drop(columns=["equipment", "location", "faulty"]):
      Q1 = data[col].quantile(0.25)
      Q3 = data[col].quantile(0.75)
      IQR = Q3 - Q1
      
      aux_outliers = data[
        (data[col] < Q1-(IQR*1.5)) |
        (data[col] > Q3+(IQR*1.5))
      ]
      
      outliers.update({col:aux_outliers.index.tolist()})
      
    return outliers
  
  # Correlation with Faulty
  @property
  def correlation(self):
    """Print correlation between numeric columns and faulty column
    """
    data = self.data
    for col in data.drop(columns=["equipment", "location", "faulty"]):
      print(
        f"Correlation {col} x faulty: {round(
          np.corrcoef(data[col], data["faulty"])[0][1], 3)}")
  
  # Correlation HeatMap
  @property
  def corr_heatmap(self):
    """Plot a HeatMap of Correlation between numeric columns
    """
    plt.figure(figsize=(12, 6))
    sns.heatmap(df.drop(columns=["equipment", "location"]).corr(), annot=True)
    plt.title(
      "Correlation".upper(),
      fontdict={'family':'calibri', 'fontsize':18, 'weight':'bold', 'color':'red'},
      pad=20
    )
    plt.show()
    
  # Plot Logistic Plot
  @property
  def logistic_plot(self):
    """ScatterPlot for all numeric columns where y=faulty"""
    data = self.data
    for col in data.drop(columns=["equipment", "location", "faulty"]):
      fig = go.Figure()
      fig.add_trace(go.Scatter(
        x=data[col],
        y=data["faulty"],
        mode='markers',
        marker_color=data["faulty"]
        )
      )
      fig.show()
    


In [4]:
# Create a class to reduce the quantity of code
class RandomForestModel:
  """Build and Train a RandomForestClassifier. Will load all class when run the code.

    Attributes:
    - X_train: 80% of Independents Variables
    - X_test: 20% of Independents Variables
    - Y_train 80% of Dependents Variables
    - Y_test: 20% of Dependents Variables
    - model: Trained Random Forest Model
    - y_pred: Predicted values
    - accuracy: Model accuracy
    - conf_matrix: Confusion Matrix of predicted values
    - class_report: Text report that shows main metrics
    - plot_conf_matrix(): Plot Confusion Matrix
    - scatter_3d_test_data(): Plot a 3D scatter of predicted values
  """
  def __init__(self, dataframe):
    self.data = dataframe
    self.X_train, self.X_test, self.Y_train, self.Y_test = self.__train_data
    self.model, self.y_pred, self.accuracy, self.conf_matrix, self.class_report = self.__best_random_forest
  
  @property
  def __train_data(self):
    data = self.data
    X = data.drop(columns=["equipment", "location", "faulty", "humidity"]).values
    Y = data["faulty"].values

    X_train, X_test, y_train, y_test = train_test_split(
      X,
      Y,
      test_size=0.2,
      random_state=42
    )
    return X_train, X_test, y_train, y_test
  
  @property
  def __best_random_forest(self):
    """Build and Train a RandomForestClassifier Model with GrindSearchCV

    Returns:
      5 variables:
        - RandomForestClassifier Model
        - Prediction
        - Accuracy
        - Confusion Matrix
        - Classification Report
    """
    
    X_train = self.X_train
    X_test = self.X_test
    y_train = self.Y_train
    y_test = self.Y_test
    rf = RandomForestClassifier()
    
    param_grid = {
      'n_estimators': [100, 150, 200],
      'max_depth': [None, 1, 5, 10],
      'min_samples_leaf': [1, 5, 10]
    }

    grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy')
    
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_

    y_pred = best_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    class_report = classification_report(y_test, y_pred)

    return best_model, y_pred, accuracy, conf_matrix, class_report
  
  # Ploting the Confusion Matrix
  def plot_conf_matrix(self):
    """Plot a Confusion Matrix

    Args:
        conf_matrix (matrix): Confusion Matrix calculated
        y_train (array): Data for Test
        title (str): Graph Title
    """
    
    conf_matrix = self.conf_matrix
    plt.figure(figsize=(10, 7))
    ax= sns.heatmap(
      conf_matrix,
      annot=True,
      fmt='d',
      cmap='Blues',
      xticklabels=[0,1],
      yticklabels=[0,1]
    )


    plt.xlabel('Predicted', fontdict={'fontsize':14}, labelpad=20)
    plt.ylabel('Actual', fontdict={'fontsize':14}, labelpad=20)
    plt.title("Confusion Matrix", fontdict={'fontsize':16})
    plt.show();


  # Create a trace for True Positive data
  def scatter_3d_test_data(self):
    """Create a Scatter Plot 3D of the prediction

    Args:
        x_test (ndarray): Array with the test data of 3 independent variables
        y_test (ndarray): Array with the test data of dependent variable
        y_pred (ndarray): Array of predicted values
    """

    X_test = self.X_test
    y_test = self.Y_test
    y_pred = self.y_pred
    
    class_TF = X_test[(y_test == 1) & (y_pred == 0)]
    class_TT = X_test[(y_test == 1) & (y_pred == 1)]
    class_FF = X_test[(y_test == 0) & (y_pred == 0)]
    class_FT = X_test[(y_test == 0) & (y_pred == 1)]

    true_positive = go.Scatter3d(
      x=class_TT[:, 0],
      y=class_TT[:, 1],
      z=class_TT[:, 2],
      mode="markers",
      marker=dict(
        size=5,
        color="blue",
        symbol="circle"
      ),
      name="True Positive"
    )

    # Create a trace for True Negative data
    true_negative = go.Scatter3d(
      x=class_FF[:, 0],
      y=class_FF[:, 1],
      z=class_FF[:, 2],
      mode="markers",
      marker=dict(
        size=5,
        color="green",
        symbol="circle"
      ),
      name="True Negative"
    )

    # Create a trace for False Positive data
    false_positive = go.Scatter3d(
      x=class_FT[:, 0],
      y=class_FT[:, 1],
      z=class_FT[:, 2],
      mode="markers",
      marker=dict(
        size=5,
        color="red",
        symbol="circle"
      ),
      name="False Positive"
    )

    # Create a trace for False Negative data
    false_negative = go.Scatter3d(
      x=class_TF[:, 0],
      y=class_TF[:, 1],
      z=class_TF[:, 2],
      mode="markers",
      marker=dict(
        size=5,
        color="orange",
        symbol="circle"
      ),
      name="False Negative"
    )

    # Configure graph layout
    layout = go.Layout(
      scene=dict(
        xaxis_title="Temperature",
        yaxis_title="Pressure",
        zaxis_title="Vibration"
      ),
      legend=dict(
        x=0,
        y=1
      ),
      width=1000,
      height=800
    )

    # Create a figure and add the traces
    fig = go.Figure(
      data=[true_positive, true_negative, false_positive, false_negative],
      layout=layout)

    fig.update_layout(
      title={
        'text':"<b>Predict Values</b>",
        'font': {
          'family':'Arial Black',
          'size': 30,
          'color': 'black'
        },
        'x': 0.5,
        'xanchor': 'center'
      },
      legend={
        'font': {
          'family':'Calibri',
          'size': 16,
          'color': 'black'
      },
        'itemsizing':'constant'} # Increase legend marker size
    )
    # Show graph
    fig.show()

# <h1 style="font-size:20px">All Equipments</h1>

In [ ]:
df.describe()

In [ ]:
df.info()

In [7]:
analysis = Analysis(df)

In [ ]:
analysis.plot_kde

In [ ]:
analysis.plot_boxplot

In [ ]:
analysis.proportion

In [ ]:
analysis.outliers["temperature"]

In [ ]:
analysis.correlation

In [ ]:
analysis.corr_heatmap

In [ ]:
analysis.logistic_plot

## <h1 style="font-size:20px">Machine Learning</h1>

In [15]:
X_train, X_test, y_train, y_test = train_test_split(
  df.drop(columns=["equipment", "location", "faulty"]).values,
  df["faulty"].values,
  test_size=0.2,
  random_state=42,
  stratify=df["faulty"].values
)

### <h1 style="font-size:20px">KNeighbors</h1>

In [16]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [17]:
knn = KNeighborsClassifier()
param_grid = {'n_neighbors': np.arange(1, 21)}
grid_search = GridSearchCV(knn, param_grid, cv=5)
grid_search.fit(X_train_scaled, y_train)
best_n_neighbors = grid_search.best_params_['n_neighbors']

In [ ]:
best_knn = KNeighborsClassifier(n_neighbors=best_n_neighbors)
best_knn.fit(X_train_scaled, y_train)

In [19]:
y_pred_knn = best_knn.predict(X_test_scaled)
accuracy_knn = accuracy_score(y_test, y_pred_knn)

In [20]:
accuracy_knn = accuracy_score(y_test, y_pred_knn)
conf_matrix_knn = confusion_matrix(y_test, y_pred_knn)
class_report_knn = classification_report(y_test, y_pred_knn)

In [ ]:
print(f"Accuracy: {accuracy_knn:.3f}")
print()

# Precision has the best accuracy
print("Classification Report:")
print(class_report_knn)

In [ ]:
# Ploting the Confusion Matrix
plt.figure(figsize=(10, 7))
ax= sns.heatmap(
  conf_matrix_knn,
  annot=True,
  fmt='d',
  cmap='Blues',
  xticklabels=set(y_train),
  yticklabels=set(y_train)
)


plt.xlabel('Predicted', fontdict={'fontsize':14}, labelpad=20)
plt.ylabel('Actual', fontdict={'fontsize':14}, labelpad=20)
plt.title('Confusion Matrix KNeighbors', fontdict={'fontsize':16})
plt.show();

### <h1 style="font-size:20px">Random Forest</h1>

In [23]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [ ]:
model_rf = RandomForestClassifier(n_estimators=150, random_state=42)
model_rf.fit(X_train_scaled, y_train)

In [25]:
y_pred_rf = model_rf.predict(X_test_scaled)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)
class_report_rf = classification_report(y_test, y_pred_rf)

In [ ]:
print(f"Accuracy: {accuracy_rf:.3f}")
print()

# Precision has the best accuracy
print("Classification Report:")
print(class_report_rf)

In [ ]:
# Ploting the Confusion Matrix
plt.figure(figsize=(10, 7))
ax= sns.heatmap(
  conf_matrix_rf,
  annot=True,
  fmt='d',
  cmap='Blues',
  xticklabels=set(y_train),
  yticklabels=set(y_train)
)


plt.xlabel('Predicted', fontdict={'fontsize':14}, labelpad=20)
plt.ylabel('Actual', fontdict={'fontsize':14}, labelpad=20)
plt.title('Confusion Matrix Random Forest', fontdict={'fontsize':16})
plt.show();

# <h1 style="font-size:20px">Each equipment</h1>

In [28]:
turbine = df[df["equipment"] == "Turbine"]
compressor = df[df["equipment"] == "Compressor"]
pump = df[df["equipment"] == "Pump"]

## <h1 style="font-size:20px">Turbine</h1>

### <h1 style="font-size:20px">Analysis</h1>

In [ ]:
turbine.describe()

In [30]:
turb_analysis = Analysis(turbine)

In [ ]:
turb_analysis.plot_kde

In [ ]:
turb_analysis.plot_boxplot

In [ ]:
turb_analysis.proportion

In [ ]:
turb_analysis.outliers["temperature"]

In [ ]:
turb_analysis.correlation

In [ ]:
turb_analysis.corr_heatmap

In [ ]:
turb_analysis.logistic_plot

### <h1 style="font-size:20px">Machine Learning</h1>

In [38]:
turbine_model = RandomForestModel(turbine)

In [ ]:
print(f"Accuracy: {turbine_model.accuracy:.3f}")
print()

# Precision has the best accuracy
print("Classification Report:")
print(turbine_model.class_report)

In [ ]:
turbine_model.plot_conf_matrix()

In [ ]:
turbine_model.scatter_3d_test_data()

## <h1 style="font-size:20px">Compressor</h1>

### <h1 style="font-size:20px">Analysis</h1>

In [ ]:
compressor.describe()

In [43]:
compr_analysis = Analysis(compressor)

In [ ]:
compr_analysis.plot_kde

In [ ]:
compr_analysis.plot_boxplot

In [ ]:
compr_analysis.proportion

In [ ]:
compr_analysis.outliers["temperature"]

In [ ]:
compr_analysis.correlation

In [ ]:
compr_analysis.corr_heatmap

In [ ]:
compr_analysis.logistic_plot

### <h1 style="font-size:20px">Machine Learning</h1>

In [51]:
compressor_model = RandomForestModel(compressor)

In [ ]:
print(f"Accuracy: {compressor_model.accuracy:.3f}")
print()

# Precision has the best accuracy
print("Classification Report:")
print(compressor_model.class_report)

In [ ]:
compressor_model.plot_conf_matrix()

In [ ]:
compressor_model.scatter_3d_test_data()

## <h1 style="font-size:20px">Pump</h1>

### <h1 style="font-size:20px">Analysis</h1>

In [ ]:
pump.describe()

In [56]:
pump_analysis = Analysis(pump)

In [ ]:
pump_analysis.plot_kde

In [ ]:
pump_analysis.plot_boxplot

In [ ]:
pump_analysis.proportion

In [ ]:
pump_analysis.outliers["temperature"]

In [ ]:
pump_analysis.correlation

In [ ]:
pump_analysis.corr_heatmap

In [ ]:
pump_analysis.logistic_plot

### <h1 style="font-size:20px">Machine Learning</h1>

In [64]:
pump_model = RandomForestModel(pump)

In [ ]:
print(f"Accuracy: {pump_model.accuracy:.3f}")
print()

# Precision has the best accuracy
print("Classification Report:")
print(pump_model.class_report)

In [ ]:
pump_model.plot_conf_matrix()

In [ ]:
pump_model.scatter_3d_test_data()